# Extract frames from videos
Run this notebook in the sequential order to extract frames from videos. 

In [1]:
# import packages
import os
import pandas as pd
import numpy as np
import cv2
from tqdm.auto import tqdm

In [2]:
# hyper-parameters 
IS_SINGLE_CAMERA = True
CAMERA_ID = 5

META_DATA_PATH = '../data/Data_Description.xlsx'
VIDEO_DATA_PATH = '../data/Fall_Simulation_Data/'
OUTPUT_FRAMES_PATH = f'../data/Frames_Extracted_Camera{CAMERA_ID}/'


FRAMES_PER_SECOND = 30

In [3]:
# load meta-data
fall_metadata = pd.read_excel(META_DATA_PATH, header=1, index_col=0, parse_dates=True)
print(fall_metadata.shape)

fall_metadata['Start.1'] = pd.to_timedelta(fall_metadata['Start.1'].astype(str).str.strip()).dt.total_seconds()
fall_metadata['Fall'] = pd.to_timedelta(fall_metadata['Fall'].astype(str).str.strip()).dt.total_seconds()
fall_metadata['End.1'] = pd.to_timedelta(fall_metadata['End.1'].astype(str).str.strip()).dt.total_seconds()
fall_metadata['Length'] = pd.to_timedelta(fall_metadata['Length'].astype(str).str.strip()).dt.total_seconds()

fall_metadata.head(5)

(55, 16)


,Speed,WA,MO,Start,End,Actor,Scenario.1,Start.1,Fall,End.1,Length,Cam1,Cam2,Cam3,Cam4,Cam5
Scenario,,,,,,,,,,,,,,,,
1,SF,Walker,Walker,Standing,Lying,Act1,Person falls backwards while walking with walk...,47.0,48.0,50.0,138.0,5,2,3,1,4
2,SF,Walker,Walker,Standing,Lying,Act1,Forward fall while walking with walker,97.0,98.0,100.0,270.0,4,1,3,2,5
3,SF,Walker,Chair,Stand/sit transition,Lying,Act1,Person falls while attempting to sit down into...,128.0,129.0,131.0,235.0,5,2,3,1,4
4,SF,Walker,na,Stand/sit transition,Lying,Act1,Person falls while attempting to sit down in sofa,95.0,94.0,96.0,235.0,4,2,3,1,5
5,FF,na,Blanket,On knees,Lying,Act2,Person is sitting on knees next to bed making ...,32.0,33.0,34.0,235.0,5,1,4,2,3


In [6]:
video_file2extracted_num = {}  # for debugging 

In [7]:
# extract the frames 
video_files = [x for x in os.listdir(VIDEO_DATA_PATH)]


prog_bar = tqdm()  # define the progress bar object
for video_file in video_files:
    prog_bar.refresh()
    prog_bar.set_description(video_file)
    
    video_path = os.path.join(VIDEO_DATA_PATH, video_file)
    file_name = video_file.split('.')[0]  # used to format the png name
    
    if video_file in video_file2extracted_num:  # skip the processed video
        continue
    if IS_SINGLE_CAMERA:  # skip the unexpected camera 
        camera_id = int(video_file.split('.')[0][-1])
        if camera_id != CAMERA_ID:
            continue 
        
    is_fall = 'Fall' in video_file
    if is_fall:  # extract the durations of fall
        scenario_id = int(video_file.split('_')[0][4:])
        print(f'DEBUG: processing Fall Scenario {scenario_id}')
        scenario_info = fall_metadata.loc[scenario_id]  # iloc selects the wrong row
        length_seconds = scenario_info['Length']
        fall_seconds = scenario_info['Fall']
        end_seconds = scenario_info['End.1']
        prog_bar.reset(total=length_seconds * FRAMES_PER_SECOND)
    else:
        prog_bar.reset(total=25 * 60 * FRAMES_PER_SECOND)  # approximate time for illustration purpose only
   
    # extract the frames
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(f'[DEBUG] FPS={fps}')
    frame_counter = 0 
    recorded_frame_num = 0 
    if is_fall:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:  # when the video is finished
                break     
            prog_bar.update(1)
            if fall_seconds <= frame_counter // FRAMES_PER_SECOND <= end_seconds:  # extract the falling frames
                if frame_counter % 3 == 0:  # extracted at 10fps
                    cv2.imwrite(os.path.join(OUTPUT_FRAMES_PATH, file_name + f'_{frame_counter}_fall.jpg'), frame)
                    recorded_frame_num += 1
            elif frame_counter // FRAMES_PER_SECOND > end_seconds:  # skip the remaining periods for faster processing
                break
            frame_counter += 1
    else:  # extract frames at low sampling rate i.e., 0.2 fps
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:  # when the video is finished
                break  
            prog_bar.update(1)
            if frame_counter % (FRAMES_PER_SECOND * 15) == 0:
                cv2.imwrite(os.path.join(OUTPUT_FRAMES_PATH, file_name + f'_{frame_counter}_unfall.jpg'), frame)
                recorded_frame_num += 1
            frame_counter += 1
    cap.release()
    cv2.destroyAllWindows() # destroy all opened windows
    
    video_file2extracted_num[video_file] = recorded_frame_num 

0it [00:00, ?it/s]

[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 10
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 11
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 12
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 13
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 14
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 15
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 16
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 17
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 18
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 19
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 1
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 20
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 21
[DEBUG] FPS=30.0
DEBUG: processing Fall Scenario 22
[

In [ ]:
video_file2extracted_num

In [ ]:
os.path.join(OUTPUT_FRAMES_PATH, file_name + f'_{frame_counter}_fall.jpg')